In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [4]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [6]:
train_loss = tf.keras.metrics.Mean()
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

test_loss = tf.keras.metrics.Mean()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [7]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [8]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [9]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}', 
    f'Loss: {train_loss.result()}', 
    f'Accuracy: {train_accuracy.result() * 100}', 
    f'Test Loss: {test_loss.result()}', 
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

2024-02-17 15:17:00.804216: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.
2024-02-17 15:17:01.708682: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-02-17 15:17:02.404625: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f6b456437c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-17 15:17:02.404654: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-02-17 15:17:02.409242: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1708170422.511577  166883 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch 1 Loss: 0.14097028970718384 Accuracy: 95.74666595458984 Test Loss: 0.0738678127527237 Test Accuracy: 97.47000122070312
Epoch 2 Loss: 0.04102122411131859 Accuracy: 98.74666595458984 Test Loss: 0.05726142227649689 Test Accuracy: 98.1500015258789
Epoch 3 Loss: 0.01896830089390278 Accuracy: 99.37999725341797 Test Loss: 0.06203995645046234 Test Accuracy: 98.27999877929688
Epoch 4 Loss: 0.012103147804737091 Accuracy: 99.58999633789062 Test Loss: 0.06752266734838486 Test Accuracy: 98.19999694824219
Epoch 5 Loss: 0.009685322642326355 Accuracy: 99.69166564941406 Test Loss: 0.07057774811983109 Test Accuracy: 98.32999420166016


In [28]:
model.save("my_model")
model = tf.keras.models.load_model("my_model")

INFO:tensorflow:Assets written to: temp_model/assets


INFO:tensorflow:Assets written to: temp_model/assets


In [36]:
import cv2
import numpy as np


img = cv2.imread("three.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (28, 28))
img = img[tf.newaxis, ...]
img = img[..., tf.newaxis].astype("float32")

output = model(img)
result = np.argmax(output)
result

3